In [3]:
!pip install selenium
# !pip install pandas
!pip install lxml

  Using cached selenium-4.20.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached trio-0.25.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached selenium-4.20.0-py3-none-any.whl (9.5 MB)
Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)
Using cached trio-0.25.0-py3-none-any.whl (467 kB)
Using cached trio_websocket-0.11.1-py3-non

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from io import StringIO
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [2]:
# 크롬드라이버 실행
driver = webdriver.Chrome()
driver.implicitly_wait(10) # seconds

# wait until someid is clickable
wait = WebDriverWait(driver, 10)

In [3]:
# 페이지 로드를 기다림
def wait_for_page_to_load(driver):
    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")
    
def select_season(driver, season):
    wait_for_page_to_load(driver)
    select_element = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlSeason_ddlSeason"]')
    select = Select(select_element)
    select.select_by_visible_text(season)
    
def select_team(driver, team):
    wait_for_page_to_load(driver)
    select_element = driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlTeam_ddlTeam')
    select = Select(select_element)
    select.select_by_value(team)
    
def select_series(driver, series):
    wait_for_page_to_load(driver)
    try:
        select_element = driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlSeries_ddlSeries')
        select = Select(select_element)
        # select.select_by_value(series)
        select.select_by_value(series)
    except:
        time.sleep(5)
        select_element = driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlSeries_ddlSeries')
        select = Select(select_element)
        # select.select_by_value(series)
        select.select_by_value(series)
    
def check_page(driver, series_num):
    wait_for_page_to_load(driver)
    if series_num == '0':
        # 2페이지 있는지 확인
        page_num = len(driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[3]/div').text.split(' '))
        return page_num
    else:
        page_num = len(driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/div').text.split(' '))
        return page_num
    
def next_page(driver):
    wait_for_page_to_load(driver)
    page = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ucPager_btnNo2"]')
    page.click()
    
def prev_page(driver):
    wait_for_page_to_load(driver)
    page = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ucPager_btnNo1"]')
    page.click()
    
# 표 불러오기
def get_table(driver, season, series_num):
    wait_for_page_to_load(driver)
    # print(series_num, type(series_num))
    # 표 찾기
    # if season < 2002:
    #     table = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/table')
    # elif series_num == '0':
    #     table = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[3]/table')
    # else:
    #     table = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/table')
        
    try:
        table = [x.split(' ') for x in driver.find_element(By.TAG_NAME, 'table').text.split('\n')]
        df = pd.DataFrame(table[1:], columns=table[0])
    except:
        time.sleep(5)
        table = [x.split(' ') for x in driver.find_element(By.TAG_NAME, 'table').text.split('\n')]
        df = pd.DataFrame(table[1:], columns=table[0])
        
    # # 테이블의 HTML을 문자열로 얻습니다.
    # table_html = table.get_attribute('outerHTML')

    # # StringIO 객체를 사용하여 HTML 문자열을 래핑합니다.
    # table_html_io = StringIO(table_html)

    # # Pandas의 read_html 함수를 사용하여 HTML 테이블을 DataFrame으로 변환합니다.
    # df_list = pd.read_html(table_html_io)

    # # 만약 여러 개의 테이블이 있다면 df_list에 여러 개의 데이터프레임이 저장됩니다.
    # # 원하는 데이터프레임을 선택합니다.
    # desired_df = df_list[0]  # 예를 들어, 첫 번째 테이블을 선택합니다.

    # # 데이터프레임을 출력하거나 필요한 작업을 수행합니다.
    # return desired_df
    return df

series_list = {
    'KBO 정규시즌': '0',
    'KBO 시범경기': '1',
    'KBO 와일드카드': '4',
    'KBO 준플레이오프': '3',
    'KBO 플레이오프': '5',
    'KBO 한국시리즈': '7'
}

team_dict = {
    '두산' : 'OB',
    '한화' : 'HH',
    '빙그레' : 'HH',
    'KIA' : 'HT',
    '해태' : 'HT',
    'NC' : 'NC',
    'LG' : 'LG',
    '청보' : 'HD',
    '현대' : 'HD',
    '쌍방울' : 'SB',
    '우리' : 'WO',
    '히어로즈': 'WO',
    '넥센' : 'WO',
    '키움' : 'WO',
    'SSG' : 'SK',
    'SK' : 'SK',
    'KT' : 'KT',
    'OB' : 'OB',
    '삼성' : 'SS',
    '삼미' : 'HD',
    'MBC' : 'LG',
    '태평양' : 'HD',
    '롯데' : 'LT'
}

In [4]:
#크롬 드라이버에 url 주소 넣고 실행
driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx')
# driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Detail1.aspx')


In [5]:
# 1984 - 2001
# base_page = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/div[1]/ul/li[1]/a')
# base_page.click()
# time.sleep(.1)

df_list = []

try:

    for season in range(2001, 2002):
        select_season(driver, str(season))
        time.sleep(.5)
        
        team_select_element = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlTeam_ddlTeam"]')
        team_select = Select(team_select_element)
        team_option = team_select.options[1:]
        time.sleep(.5)

        team_list = [team.text for team in team_option]

        for team in team_list:
            select_team(driver, team_dict[team])
            time.sleep(.5)
        
            for key, series in series_list.items():
                select_series(driver, series)
                
                time.sleep(.5)
                df = get_table(driver, season=season)
                df['년도'] = season
                df['팀명'] = team
                df['경기타입'] = key
                
                df_list.append(df)
                print(season, team, key, df.shape, '완료')

except:
    time.sleep(180)
    driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx')
    time.sleep(.5)
    
    season_pre = season
    
    for season in range(season_pre, 2001):
        select_season(driver, str(season))
        time.sleep(.5)
        
        team_select_element = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlTeam_ddlTeam"]')
        team_select = Select(team_select_element)
        team_option = team_select.options[1:]
        time.sleep(.5)

        team_list = [team.text for team in team_option]

        for team in team_list:
            select_team(driver, team_dict[team])
            time.sleep(.5)
        
            for key, series in series_list.items():
                select_series(driver, series)
                
                time.sleep(.5)
                df = get_table(driver, season=season)
                df['년도'] = season
                df['팀명'] = team
                df['경기타입'] = key
                
                df_list.append(df)
                print(season, team, key, df.shape, '완료')


# if check_page(driver) > 1:
#     next_page(driver)
    


2001 두산 KBO 정규시즌 (30, 21) 완료
2001 두산 KBO 와일드카드 (0, 21) 완료
2001 두산 KBO 준플레이오프 (13, 21) 완료
2001 두산 KBO 플레이오프 (13, 21) 완료
2001 두산 KBO 한국시리즈 (16, 21) 완료
2001 삼성 KBO 정규시즌 (30, 21) 완료
2001 삼성 KBO 와일드카드 (0, 21) 완료
2001 삼성 KBO 준플레이오프 (0, 21) 완료
2001 삼성 KBO 플레이오프 (0, 21) 완료
2001 삼성 KBO 한국시리즈 (15, 21) 완료
2001 현대 KBO 정규시즌 (28, 21) 완료
2001 현대 KBO 와일드카드 (0, 21) 완료
2001 현대 KBO 준플레이오프 (0, 21) 완료
2001 현대 KBO 플레이오프 (11, 21) 완료
2001 현대 KBO 한국시리즈 (0, 21) 완료
2001 한화 KBO 정규시즌 (30, 21) 완료
2001 한화 KBO 와일드카드 (0, 21) 완료
2001 한화 KBO 준플레이오프 (16, 21) 완료
2001 한화 KBO 플레이오프 (0, 21) 완료
2001 한화 KBO 한국시리즈 (0, 21) 완료
2001 KIA KBO 정규시즌 (30, 21) 완료
2001 KIA KBO 와일드카드 (0, 21) 완료
2001 KIA KBO 준플레이오프 (0, 21) 완료
2001 KIA KBO 플레이오프 (0, 21) 완료
2001 KIA KBO 한국시리즈 (0, 21) 완료
2001 LG KBO 정규시즌 (30, 21) 완료
2001 LG KBO 와일드카드 (0, 21) 완료
2001 LG KBO 준플레이오프 (0, 21) 완료
2001 LG KBO 플레이오프 (0, 21) 완료
2001 LG KBO 한국시리즈 (0, 21) 완료
2001 SK KBO 정규시즌 (30, 21) 완료
2001 SK KBO 와일드카드 (0, 21) 완료
2001 SK KBO 준플레이오프 (0, 21) 완료
2001 SK KBO 플레이오프 (0, 21)

In [190]:
new_df = pd.concat(df_list)
new_df

,순위,선수명,팀명,AVG,G,PA,AB,H,2B,3B,...,RBI,SB,CS,BB,HBP,SO,GDP,E,년도,경기타입
0,1,박종호,OB,0.5,4,5,4,2,0,0,...,2,0,0,1,0,0,1,2,1982,KBO 정규시즌
1,2,윤동균,OB,0.342,77,334,284,97,20,5,...,47,9,2,45,1,27,3,2,1982,KBO 정규시즌
2,3,신경식,OB,0.334,78,320,293,98,21,3,...,54,14,8,20,3,17,2,6,1982,KBO 정규시즌
3,4,구재서,OB,0.333,34,3,3,1,0,0,...,0,1,1,0,0,0,0,0,1982,KBO 정규시즌
4,5,김우열,OB,0.31,62,255,210,65,8,0,...,41,14,4,38,6,25,0,0,1982,KBO 정규시즌
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26,심성보,SK,0.152,20,49,46,7,2,1,...,2,0,1,2,1,14,3,1,2000,KBO 정규시즌
26,27,장광호,SK,0.136,12,23,22,3,0,0,...,3,0,0,0,0,6,1,0,2000,KBO 정규시즌
27,28,장재명,SK,0.077,9,15,13,1,0,0,...,1,0,1,1,0,4,0,0,2000,KBO 정규시즌
28,29,김선섭,SK,0.05,35,22,20,1,1,0,...,1,0,1,1,1,8,1,2,2000,KBO 정규시즌


In [192]:
new_df[new_df['경기타입'] == 'KBO 한국시리즈']

,순위,선수명,팀명,AVG,G,PA,AB,H,2B,3B,...,RBI,SB,CS,BB,HBP,SO,GDP,E,년도,경기타입
0,1,정혁진,OB,1.000,2,2,2,2,1,0,...,2,0,0,0,0,0,0,0,1982,KBO 한국시리즈
1,2,윤동균,OB,0.407,6,30,27,11,3,0,...,1,0,1,3,0,2,0,0,1982,KBO 한국시리즈
2,3,김유동,OB,0.400,6,28,25,10,0,0,...,12,0,1,2,0,3,0,0,1982,KBO 한국시리즈
3,4,김우열,OB,0.250,6,30,28,7,0,0,...,6,0,0,2,0,1,0,0,1982,KBO 한국시리즈
4,5,구천서,OB,0.238,6,27,21,5,1,0,...,1,1,2,5,0,5,2,3,1982,KBO 한국시리즈
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,10,우즈,두산,0.136,7,28,22,3,0,0,...,5,0,0,6,0,12,0,1,2000,KBO 한국시리즈
10,11,최훈재,두산,0.125,5,9,8,1,0,0,...,1,0,0,1,0,1,0,0,2000,KBO 한국시리즈
11,12,안경현,두산,0.000,4,11,10,0,0,0,...,0,0,1,0,1,2,0,0,2000,KBO 한국시리즈
12,12,전상렬,두산,0.000,4,1,1,0,0,0,...,0,0,0,0,0,0,0,0,2000,KBO 한국시리즈


In [193]:
new_df.to_csv('./1982_2001_야구데이터.csv')

In [12]:
new_df

,순위,선수명,팀명,AVG,G,PA,AB,H,2B,3B,...,RBI,SB,CS,BB,HBP,SO,GDP,E,년도,경기타입
0,1,박종호,OB,0.5,4,5,4,2,0,0,...,2,0,0,1,0,0,1,2,1982,KBO 정규시즌
1,2,윤동균,OB,0.342,77,334,284,97,20,5,...,47,9,2,45,1,27,3,2,1982,KBO 정규시즌
2,3,신경식,OB,0.334,78,320,293,98,21,3,...,54,14,8,20,3,17,2,6,1982,KBO 정규시즌
3,4,구재서,OB,0.333,34,3,3,1,0,0,...,0,1,1,0,0,0,0,0,1982,KBO 정규시즌
4,5,김우열,OB,0.31,62,255,210,65,8,0,...,41,14,4,38,6,25,0,0,1982,KBO 정규시즌
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26,이대현,두산,0.056,21,19,18,1,1,0,...,1,0,0,1,0,5,0,2,2001,KBO 정규시즌
26,27,김성균,두산,0.000,11,8,8,0,0,0,...,0,0,0,0,0,4,0,1,2001,KBO 정규시즌
27,27,박명환,두산,-,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2001,KBO 정규시즌
28,27,이상훈,두산,-,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2001,KBO 정규시즌


In [18]:
new_df = pd.concat(df_list)
result_df = pd.concat([new_df, pd.read_csv('1982_2001_야구데이터.csv', index_col=0)])

In [19]:
result_df.to_csv('./1982_2001_야구데이터_2.csv')

In [20]:
result_df

,순위,선수명,팀명,AVG,G,PA,AB,H,2B,3B,...,RBI,SB,CS,BB,HBP,SO,GDP,E,년도,경기타입
0,1,심재학,두산,0.344,117,469,369,127,20,1,...,88,7,1,86,7,50,6,6,2001,KBO 정규시즌
1,2,김태형,두산,0.333,6,3,3,1,0,0,...,0,0,0,0,0,1,0,0,2001,KBO 정규시즌
2,3,김동주,두산,0.324,103,421,364,118,18,0,...,62,2,2,47,4,57,10,3,2001,KBO 정규시즌
3,4,정수근,두산,0.306,122,546,467,143,21,9,...,53,52,11,71,1,51,8,6,2001,KBO 정규시즌
4,5,강규철,두산,0.300,8,10,10,3,0,0,...,3,0,0,0,0,2,0,1,2001,KBO 정규시즌
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26,심성보,SK,0.152,20,49,46,7,2,1,...,2,0,1,2,1,14,3,1,2000,KBO 정규시즌
26,27,장광호,SK,0.136,12,23,22,3,0,0,...,3,0,0,0,0,6,1,0,2000,KBO 정규시즌
27,28,장재명,SK,0.077,9,15,13,1,0,0,...,1,0,1,1,0,4,0,0,2000,KBO 정규시즌
28,29,김선섭,SK,0.05,35,22,20,1,1,0,...,1,0,1,1,1,8,1,2,2000,KBO 정규시즌


In [59]:
# 2002 - 2023
driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx')

df_list = []


for season in range(2002, 2024):
    select_season(driver, str(season))
    time.sleep(.5)
    
    team_select_element = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlTeam_ddlTeam"]')
    team_select = Select(team_select_element)
    team_option = team_select.options[1:]
    time.sleep(.5)

    team_list = [team.text for team in team_option]

    for team in team_list:
        select_team(driver, team_dict[team])
        time.sleep(.5)
    
        for key, series in series_list.items():
            select_series(driver, series)
            
            df = get_table(driver, season=season, series_num=series)
            df['년도'] = season
            df['팀명'] = team
            df['경기타입'] = key
            df_list.append(df)
            print(season, team, key, df.shape, '완료')
            time.sleep(.5)
            
            if check_page(driver, series_num=series) > 1:
                next_page(driver)
            
                time.sleep(.5)
                
                df = get_table(driver, season=season, series_num=series)
                df['년도'] = season
                df['팀명'] = team
                df['경기타입'] = key
                
                time.sleep(.5)
                
                prev_page(driver)
            
                df_list.append(df)
                print(season, team, key, df.shape, '완료_2')

2002 삼성 KBO 정규시즌 (30, 18) 완료
2002 삼성 KBO 정규시즌 (3, 18) 완료_2
2002 삼성 KBO 시범경기 (3, 18) 완료
2002 삼성 KBO 와일드카드 (21, 21) 완료
2002 삼성 KBO 준플레이오프 (0, 21) 완료
2002 삼성 KBO 플레이오프 (0, 21) 완료
2002 삼성 KBO 한국시리즈 (0, 21) 완료
2002 LG KBO 정규시즌 (17, 21) 완료
2002 LG KBO 정규시즌 (8, 18) 완료_2
2002 LG KBO 시범경기 (8, 18) 완료
2002 LG KBO 와일드카드 (8, 18) 완료
2002 LG KBO 준플레이오프 (0, 21) 완료
2002 LG KBO 플레이오프 (10, 21) 완료
2002 LG KBO 한국시리즈 (14, 21) 완료
2002 KIA KBO 정규시즌 (0, 21) 완료
2002 KIA KBO 시범경기 (29, 18) 완료
2002 KIA KBO 와일드카드 (24, 21) 완료
2002 KIA KBO 준플레이오프 (0, 21) 완료
2002 KIA KBO 플레이오프 (0, 21) 완료
2002 KIA KBO 한국시리즈 (15, 21) 완료
2002 현대 KBO 정규시즌 (0, 21) 완료
2002 현대 KBO 시범경기 (29, 18) 완료
2002 현대 KBO 와일드카드 (19, 21) 완료
2002 현대 KBO 준플레이오프 (0, 21) 완료
2002 현대 KBO 플레이오프 (14, 21) 완료
2002 현대 KBO 한국시리즈 (0, 21) 완료
2002 두산 KBO 정규시즌 (0, 21) 완료
2002 두산 KBO 시범경기 (29, 18) 완료
2002 두산 KBO 와일드카드 (21, 21) 완료
2002 두산 KBO 준플레이오프 (0, 21) 완료
2002 두산 KBO 플레이오프 (0, 21) 완료
2002 두산 KBO 한국시리즈 (0, 21) 완료
2002 SK KBO 정규시즌 (0, 21) 완료
2002 SK KBO 시범경기 (30, 18) 완료

In [61]:
pd.concat(df_list).to_csv('2002_2023_야구데이터.csv')

In [65]:
# 2002 - 2023(2번째 기록)
driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Basic2.aspx')

df_list = []


for season in range(2002, 2024):
    select_season(driver, str(season))
    time.sleep(.5)
    
    team_select_element = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlTeam_ddlTeam"]')
    team_select = Select(team_select_element)
    team_option = team_select.options[1:]
    time.sleep(.5)

    team_list = [team.text for team in team_option]

    for team in team_list:
        select_team(driver, team_dict[team])
        time.sleep(.5)
    
        for key, series in series_list.items():
            select_series(driver, series)
            
            df = get_table(driver, season=season, series_num=series)
            df['년도'] = season
            df['팀명'] = team
            df['경기타입'] = key
            df_list.append(df)
            print(season, team, key, df.shape, '완료')
            time.sleep(.5)
            
            if check_page(driver, series_num=series) > 1:
                next_page(driver)
            
                time.sleep(.5)
                
                df = get_table(driver, season=season, series_num=series)
                df['년도'] = season
                df['팀명'] = team
                df['경기타입'] = key
                
                time.sleep(.5)
                
                prev_page(driver)
            
                df_list.append(df)
                print(season, team, key, df.shape, '완료_2')

2002 LG KBO 정규시즌 (30, 17) 완료
2002 LG KBO 정규시즌 (8, 17) 완료_2
2002 LG KBO 시범경기 (8, 17) 완료
2002 LG KBO 와일드카드 (25, 21) 완료
2002 LG KBO 준플레이오프 (0, 21) 완료
2002 LG KBO 플레이오프 (10, 21) 완료
2002 LG KBO 한국시리즈 (14, 21) 완료


NoSuchElementException: Message: Cannot locate option with value: KT; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
